In [1]:
%run base.ipynb

In [2]:
experiment = 'few-shot-seeds-mulneg'


results_path = get_experiment_path(experiment)

In [3]:
results_path

PosixPath('/home/ij/data/delib-results/results/few-shot-seeds-mulneg')

In [4]:
import pandas as pd
import numpy as np
import re

def read_results(name):
    paths = list(results_path.glob(f'*/results/{name}.json'))
    dfs = []
    for path in paths:
    #     print(path)
        df = pd.read_json(path, orient='index')
        if len(df.columns) == 1:
            df = pd.DataFrame([df.to_dict()[0]], columns=df.to_dict()[0].keys())
#         print(df)
        df[['template','per_map_size', 'map_size', 'seed']] = re.findall(r'-([a-zA-Z_]*)-n(\d+)in(\d+).*seed(\d+)', str(path))
        df[['per_map_size', 'map_size', 'seed']] = df[['per_map_size', 'map_size', 'seed']].apply(lambda x: int(x))

#         display(df)
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    df.loc[df['template']=='mulneg', 'template']=''
    # display(df)
    df = pd.pivot_table(df, index=['per_map_size', 'map_size','template'], values=['p1', 'p5', 'mrr'],
                                aggfunc=[np.mean, np.std, np.count_nonzero], dropna=False)
    df = df[df['count_nonzero']['p1']==5]
    # display(df)

    formatter = lambda x: re.sub('^0\.', '.', (x[0] if type(x[0])==str else f'{x[0]:.4f}'+(f'\u00B1{x[1]:.4f}' if not pd.isna(x[1]) else '')).replace('%',''))
    f_df = pd.DataFrame({col: zip(df['mean'][col], df['std'][col]) for col in df['mean'].columns})
    f_df.index = df.index
    f_df = f_df.applymap(formatter)
    
    f_df = f_df[['p1', 'p5', 'mrr']]
#     f_df.reset_index(level=0, inplace=True)
#     f_df.columns = f_df.columns.droplevel(2)
    return f_df

In [5]:
results_df = pd.concat([read_results('avg'), read_results('annotated_samples_metrics')], keys=['test', 'samples'], axis=1)
# results_df.columns = results_df.columns.droplevel(0)
# results_df.reset_index(level=0)
results_df

test                              \
                                           p1            p5           mrr   
per_map_size map_size template                                              
8            8                   .2540±0.0034  .5539±0.0040  .3947±0.0033   
                      beginning  .2613±0.0044  .5547±0.0054  .3998±0.0051   
                      pro_con    .2700±0.0065  .5680±0.0081  .4093±0.0074   
             64                  .2733±0.0037  .5763±0.0052  .4142±0.0042   
                      beginning  .2805±0.0016  .5835±0.0027  .4229±0.0019   
                      pro_con    .2834±0.0035  .5906±0.0050  .4261±0.0036   

                                      samples                              
                                           p1            p5           mrr  
per_map_size map_size template                                             
8            8                   .4380±0.0125  .9100±0.0035  .6350±0.0072  
                      beginning  .4430±0.0091  .9110±0.0074  .6306±0.0065  
                      pro_con    .4430±0.0168  .9100±0.0141  .6322±0.0112  
             64                  .4510±0.0134  .9160±0.0129  .6482±0.0064  
                      beginning  .4760±0.0074  .9240±0.0082  .6603±0.0038  
                      pro_con    .4880±0.0115  .9240±0.0065  .6654±0.0076

In [6]:
print(results_df.to_latex())

\begin{tabular}{lllllllll}
\toprule
  &    &         & \multicolumn{3}{l}{test} & \multicolumn{3}{l}{samples} \\
  &    &         &            p1 &            p5 &           mrr &            p1 &            p5 &           mrr \\
per\_map\_size & map\_size & template &               &               &               &               &               &               \\
\midrule
8 & 8  &         &  .2540±0.0034 &  .5539±0.0040 &  .3947±0.0033 &  .4380±0.0125 &  .9100±0.0035 &  .6350±0.0072 \\
  &    & beginning &  .2613±0.0044 &  .5547±0.0054 &  .3998±0.0051 &  .4430±0.0091 &  .9110±0.0074 &  .6306±0.0065 \\
  &    & pro\_con &  .2700±0.0065 &  .5680±0.0081 &  .4093±0.0074 &  .4430±0.0168 &  .9100±0.0141 &  .6322±0.0112 \\
  & 64 &         &  .2733±0.0037 &  .5763±0.0052 &  .4142±0.0042 &  .4510±0.0134 &  .9160±0.0129 &  .6482±0.0064 \\
  &    & beginning &  .2805±0.0016 &  .5835±0.0027 &  .4229±0.0019 &  .4760±0.0074 &  .9240±0.0082 &  .6603±0.0038 \\
  &    & pro\_con &  .2834±0.0035 &  .59